In [1]:
import numpy as np
import pandas as pd

import json,re,os,time,random
from tqdm import tqdm
from datetime import datetime
from collections import defaultdict, Counter

import warnings
warnings.filterwarnings('ignore')

# 水稻处理

In [2]:
df1 = pd.read_csv('data/水稻数据.csv')
# ['省份','品种名称','亲本来源','类型','原产地','审定编号']
df1.head()

,省份,品种名称,亲本来源,类型,原产地,审定编号
0,福建,榕夏两优676,榕夏S×福恢676,籼型两系杂交稻,福建省农业科学院水稻研究所,闽审稻20210013
1,福建,澜优粤禾丝苗,澜达A×粤禾丝苗,籼型三系杂交稻,福建省农业科学院水稻研究所等,闽审稻20190031
2,福建,元两优1725,元亨S×福恢1725,籼型两系杂交稻,福建省农业科学院水稻研究所,闽审稻20210012
3,福建,野香优639,野香A×福恢639,籼型三系杂交稻,福建禾丰种业股份有限公司等,闽审稻20190030
4,福建,旌3优164,旌3A×明恢164,籼型三系杂交稻,福建农乐种业有限公司等,闽审稻20210011


In [3]:
# 删除 \u3000
df1['省份'] = df1['省份'].apply(lambda x: x.replace('\u3000',''))
df1['省份'].unique()
# 筛选数据  筛选省份是中南地区，即河南、湖北、湖南、广东、广西、海南的数据 ，删除亲本来源 包含 ? 数据
pro_list = '河南、湖北、湖南、广东、广西、海南'
pro_list = pro_list.split('、')
#pro_list
df1 = df1[df1['省份'].isin(pro_list)]

In [4]:
# 抽取审定方 中文
# 审定方： 湘品审—>湘审稻，桂审证字—>桂审稻，豫籼—>豫审稻，豫粳—>豫审稻，豫糯—>豫审稻，豫籼—>豫审稻
def extract_chinese(string):
    """
    string = "重出江湖hello的地H方。。的,world"
    extract_chinese(string)
    """
    
    import re
    # string = "重出江湖hello的地H方。。的,world"
    
    str_new = re.sub("[A-Za-z0-9\,\。]", "", string).replace(' ','')
    # 输出：神的孩子在唱歌
    #     print(str_new)
    return str_new

df1['审定方'] = df1['审定编号'].apply(extract_chinese)
df1['审定方'].unique()

array(['湘审稻', '湘品审第号', '豫审稻', '鄂审稻', '湘审稻第号', '-', '/', '豫籼号', '豫粳号', '',
       '荆粳号', '扬稻号', '四喜粘', '鄂审稻-', '香稻丸', '新乡糯号', '红南', '桂朝号', '广二矮',
       '粤审稻', '珍珠矮', '新稻-', '南京', '八四矮', '优', '香宝号', '豫糯号', '黄金晴', '特青号',
       '桂审稻号', '琼审稻', '桂审证字第号', '琼审稻（亲本）'], dtype=object)

In [5]:
# 替换名字 在里面的就替换  不在里面的空值
# 河南、湖北、湖南、广东、广西、海南
pro_list = ['豫','鄂','湘','粤','桂','琼',]

def fun(string):
    for k in pro_list:
        if k in string:
            return k+'审稻'
#     print(string,None)
    return None

df1['审定方'] = df1['审定方'].apply(fun)

In [6]:
# 审定序号：从审定编号 提取数字
def extract_num(string):
    """
    string = "重出江湖123的地H方。4。的,world"
    extract_num(string)
    # 输出 1234
    """
    
    import re
    str_new = re.sub("[A-Za-z\u4e00-\u9fa5\,\。]", "", string)
    return str_new
df1['审定序号'] = df1['审定编号'].apply(extract_num)

In [7]:
# 母本：亲本来源（"×"前为母本）
def fun(string):
    return string.split('×')[0]
    pass
df1['母本'] = df1['亲本来源'].apply(fun)

In [8]:
# 审定公司：原产地 删除"等"
df1['审定公司'] = df1['原产地'].apply(lambda x: x.replace('等',''))

In [9]:
# 去空
df1 = df1.dropna()
# 去重复
df1 = df1.drop_duplicates()
# 删除亲本来源 包含 ? 数据
df1 = df1[df1['亲本来源']!= '?']

df1.to_csv('data/水稻数据_清洗.csv',index=None)

# 农产品

In [10]:
# 分类 品名 最高价 平均价  最低价 单位 规格 产地 发布日期 
columns = ['categoryName', 'productName', 'priceMax', 'priceAvg','priceMin',
           'unit', 'productSize',  'productPlace', 'recordDate', ]

df2 = pd.read_csv('data/农产品数据.csv')
df2.head()

,categoryName,productName,priceMax,priceAvg,priceMin,unit,productSize,productPlace,recordDate
0,副食,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1636992000000
1,副食,薄千张,8.6,8.5,8.4,公斤,普通,河南,1742576861000
2,副食,薄千张,8.6,8.5,8.4,公斤,普通,河南,1742490436000
3,副食,薄千张,8.6,8.5,8.4,公斤,普通,河南,1742403693000
4,副食,薄千张,8.6,8.5,8.4,公斤,普通,河南,1742317729000


In [11]:
# 删除非中南 和排序
agri_provinces = '河南、湖北、湖南、广东、广西、海南'.split('、') 

def fun(string):
    l = str(string).split('/')  # 强制转字符串处理可能的非字符串数据
    l = [i for i in l if i in agri_provinces]  # 筛选符合条件的省份
    return '/'.join(sorted(l))  # 排序后拼接

df2['productPlace'] = df2['productPlace'].apply(fun)
df2 = df2[df2['productPlace'] != '']  # 过滤空字符串结果

In [12]:
# 没有空值  不需要处理
df2.isnull().sum()

categoryName     0
productName      0
priceMax        18
priceAvg        18
priceMin        18
unit             0
productSize      0
productPlace     0
recordDate       0
dtype: int64

In [13]:
#1708963200000
import datetime
import pandas as pd

def timestamp_to_time(timestamp):
    # 处理 Pandas.Timestamp 类型
    if isinstance(timestamp, pd.Timestamp):
        timestamp_num = int(timestamp.timestamp())  # 直接获取 Timestamp 的秒级时间戳
    else:
        timestamp_num = int(timestamp) // 1000  # 处理普通数值类型
    date = datetime.datetime.fromtimestamp(timestamp_num)
    date_str = date.strftime("%Y-%m-%d %H:%M:%S")
    return date_str

df2['recordDate'] = df2['recordDate'].apply(timestamp_to_time)

In [14]:
# 去重复
df2 = df2.drop_duplicates()
df2.to_csv('data/农产品数据_清洗.csv',index=None)

In [15]:
sorted(pro_list)

['桂', '湘', '琼', '粤', '豫', '鄂']